In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

In [9]:
base_path = '/Users/udaisingh/Downloads/kdd-cup-2013-author-disambiguation/dataRev2/'

In [11]:
paths = [os.path.join(base_path, x) for x in os.listdir(base_path)]

In [16]:
data = [pd.read_csv(x, nrows = 5) for x in paths]

In [18]:
for i in paths:
    print(i)

/Users/udaisingh/Downloads/kdd-cup-2013-author-disambiguation/dataRev2/Valid.csv
/Users/udaisingh/Downloads/kdd-cup-2013-author-disambiguation/dataRev2/PaperAuthor.csv
/Users/udaisingh/Downloads/kdd-cup-2013-author-disambiguation/dataRev2/Author.csv
/Users/udaisingh/Downloads/kdd-cup-2013-author-disambiguation/dataRev2/Journal.csv
/Users/udaisingh/Downloads/kdd-cup-2013-author-disambiguation/dataRev2/Conference.csv
/Users/udaisingh/Downloads/kdd-cup-2013-author-disambiguation/dataRev2/Train.csv
/Users/udaisingh/Downloads/kdd-cup-2013-author-disambiguation/dataRev2/Paper.csv


In [15]:
[display(x) for x in data]

,AuthorId,PaperIds
0,55,2507 15471 19294 20444 24074 25103 25304 36459...
1,426,873 37257 92584 174625 212235 284639 581263 60...
2,1861,59550 124595 282043 528639 922192 1048827 1059...
3,10212,24013 49767 131299 173546 195044 199989 213826...
4,10275,300441 358576 368138 389312 871240 876311 9952...


,PaperId,AuthorId,Name,Affiliation
0,1,521630,Ayman Kaheel,Cairo Microsoft Innovation Lab
1,1,972575,Mahmoud Refaat,Cairo Microsoft Innovation Lab
2,1,1528710,Ahmed Abdul-hamid,Cairo Microsoft Innovation Lab
3,1,1611750,Motaz Ahmad El-saban,Cairo Microsoft Innovation Lab
4,2,1682088,Julio D. Rossi,"Departamento Matemática, FCEyN UBA (1428) Buen..."


,Id,Name,Affiliation
0,9,Ernest Jordan,NaN
1,14,K. MORIBE,NaN
2,15,D. Jakominich,NaN
3,25,William H. Nailon,NaN
4,37,P. B. Littlewood,Cavendish Laboratory|Cambridge University


,Id,ShortName,FullName,HomePage
0,1,ICOM,Zeitschrift Für Interaktive Und Kooperative Me...,http://www.i-com-media.de
1,2,AEPIA,"Inteligencia Artificial,revista Iberoamericana...",http://aepia.dsic.upv.es/revista/
2,3,IBMRD,Ibm Journal of Research and Development,http://www-tr.watson.ibm.com/journal/rdindex.html
3,4,IBMSJ,Ibm Systems Journal,http://researchweb.watson.ibm.com/journal/
4,5,NaN,Iet Software/iee Proceedings - Software,http://www.ietdl.org/IET-SEN


,Id,ShortName,FullName,HomePage
0,1,IADIS,International Association for Development of t...,NaN
1,2,IADT,Issues and Applications of Database Technology,http://www.informatik.uni-trier.de/~ley/db/con...
2,4,NaN,IBM Germany Scientific Symposium Series,http://www.informatik.uni-trier.de/~ley/db/con...
3,5,ICOMP,International Conference on Internet Computing,http://www.informatik.uni-trier.de/~ley/db/con...
4,6,ICAC,International Conference on Autonomic Computing,http://www.autonomic-conference.org/


,AuthorId,ConfirmedPaperIds,DeletedPaperIds
0,826,25733 47276 77012 79468 87141 101385 104556 11...,220293 379873 384468 565252 939749 944658 1164...
1,933,1739240,477879
2,1118,49963 93433 341015 415282 488635 517119 521922...,961444 1446566
3,2783,154377 212636 334024 350747 696269 704564 1241...,208521 1207662 1898210
4,3105,603562 647574 792910 844605 863071 878798 1082...,191079 226980 273592 529834 556687 1052455 131...


,Id,Title,Year,ConferenceId,JournalId,Keyword
0,1,Stitching videos streamed by mobile phones in ...,2009,167,0,mobile video capturing|real-time|video stitching
1,2,A nonlocal convection–diffusion equation,2007,0,7234,Nonlocal diffusion; Convection–diffusion; Asym...
2,3,Area Effects in Cepaea,1963,0,16867,NaN
3,4,Multiple paternity in a natural population of ...,2005,0,6130,NaN
4,5,Complexity of Finding Short Resolution Proofs,1997,158,0,NaN


[None, None, None, None, None, None, None]

In [19]:
class graphEmbedder:
    
    def __init__(self):
        self.entityNodes = set()
        self.Graphs = []
    
    def defineKeys(self,df,idCol):
        temp = set(df[idCol])
        self.entityNodes.update(temp)
    
    def embeddOrdinal(self,dfs,idCol,column,name,bins=10, equalBinSize = True):
        df = pd.concat([df[[idCol,column]] for df in dfs])
        if equalBinSize: temp = pd.qcut(df[column], q = bins)
        else: temp = temp = pd.cut(df[column], bins = bins)
        
        tempDf = df[[idCol]].copy(deep=True)
        tempDf['temp'] = temp
        graph = dict(tempDf.dropna().to_numpy())
        self.Graphs.append((name,graph))
        return True
    
    def embeddText(self,dfs,idCol, column,name, min_df = 0.01, method = 'BagOfWords'):
        df = pd.concat([df[[idCol,column]] for df in dfs])
        temp = df[[idCol, column]]
        temp = temp.dropna()
        if method == 'BagOfWords': vectorizer = CountVectorizer(min_df=min_df, ngram_range = (1,2))
        elif method == 'TFIDF': vectorizer = TfidfVectorizer(min_df=min_df, ngram_range = (1,2))
        
        output = CV.fit_transform(temp[column]).toarray()
        tokens = CV.get_feature_names()
        idVal = list(temp[idCol])
        
        graph = {}
        for i in range(output.shape[0]):
            words = []
            for j in range(output.shape[1]):
                if output[i][j] > 0:
                    words.append(tokens[j])
            graph[idVal[i]] = words
        
        self.Graphs.append((name,graph))
        return True
    
    def embeddCategorical(self,dfs,idCol,column,name):
        df = pd.concat([df[[idCol,column]] for df in dfs])
        graph = dict(df[[idCol,column]].to_numpy())
        self.Graphs.append((name,graph))
        return True
    
    def defineTruth(self, df):
        graph = dict(df.to_numpy())
        self.Graphs.append(('ground_truth',graph))
        return True
    
    def saveGraph(self, method = 'csv', fname = os.path.join(os.getcwd(),'heteroGraph.csv')):
        df = pd.DataFrame(columns = ['source', 'target','type'])
        for graph in self.Graphs:
            name = graph[0]
            links = graph[1]
            data = self._graphFixer(links)
            temp = pd.DataFrame(data = data, columns = ['source', 'target'])
            temp['type'] = name
            
            df = pd.concat([df, temp])
        df.to_csv(os.path.join(os.getcwd(),fname), index = False)
    
    def _graphFixer(self, dictionary):
        temp = []
        for key, value in dictionary.items():
            if type(value)==list:
                for v in value:
                    temp.append((key,v))
            else:
                temp.append((key,value))
        return temp

## Abt-Buy Dataset

In [20]:
def fixPrice(x):
    if not (type(x)== float):
        try:
            return re.findall("\d+\.\d+", x)[0]
        except:
            return np.nan
    else:
        return x

In [21]:
pt1 = "../data/Abt-Buy/Abt.csv"
pt2 = "../data/Abt-Buy/Buy.csv"
pt3 = "../data/Abt-Buy/abt_buy_perfectMapping.csv"

Abt = pd.read_csv(pt1, engine = 'python')
Buy = pd.read_csv(pt2, engine = 'python')
truth = pd.read_csv(pt3)

Abt['price'] = pd.to_numeric(Abt.price.apply(fixPrice))
Buy['price'] = pd.to_numeric(Buy.price.apply(fixPrice))

In [22]:
display(Abt.head())
display(Buy.head())

,id,name,description,price
0,552,Sony Turntable - PSLX350H,Sony Turntable - PSLX350H/ Belt Drive System/ ...,NaN
1,580,Bose Acoustimass 5 Series III Speaker System -...,Bose Acoustimass 5 Series III Speaker System -...,399.0
2,4696,Sony Switcher - SBV40S,Sony Switcher - SBV40S/ Eliminates Disconnecti...,49.0
3,5644,Sony 5 Disc CD Player - CDPCE375,Sony 5 Disc CD Player- CDPCE375/ 5 Disc Change...,NaN
4,6284,Bose 27028 161 Bookshelf Pair Speakers In Whit...,Bose 161 Bookshelf Speakers In White - 161WH/ ...,158.0


,id,name,description,manufacturer,price
0,10011646,Linksys EtherFast EZXS88W Ethernet Switch - EZ...,Linksys EtherFast 8-Port 10/100 Switch (New/Wo...,LINKSYS,NaN
1,10140760,Linksys EtherFast EZXS55W Ethernet Switch,5 x 10/100Base-TX LAN,LINKSYS,NaN
2,10221960,Netgear ProSafe FS105 Ethernet Switch - FS105NA,NETGEAR FS105 Prosafe 5 Port 10/100 Desktop Sw...,Netgear,NaN
3,10246269,Belkin Pro Series High Integrity VGA/SVGA Moni...,1 x HD-15 - 1 x HD-15 - 10ft - Beige,Belkin,NaN
4,10315184,Netgear ProSafe JFS516 Ethernet Switch,Netgear ProSafe 16 Port 10/100 Rackmount Switc...,Netgear,NaN


In [25]:
gE = graphEmbedder()

In [31]:
pd.cut(Abt.price, bins = 5, labels = False)

0       NaN
1       1.0
2       0.0
3       NaN
4       0.0
       ... 
1076    0.0
1077    2.0
1078    0.0
1079    NaN
1080    0.0
Name: price, Length: 1081, dtype: float64

In [473]:
gE.defineKeys(Abt, 'id')
gE.defineKeys(Buy, 'id')

In [474]:
gE.defineTruth(truth)

True

In [475]:
gE.embeddText([Abt,Buy], 'id','name','name', min_df=0)

True

In [476]:
gE.embeddText([Abt,Buy], 'id','description','description', min_df = 0.05)

True

In [477]:
gE.embeddOrdinal([Abt,Buy],'id','price','price', bins = 5)

True

In [478]:
gE.embeddCategorical([Buy], 'id', 'manufacturer', 'manufacturer')

True

In [479]:
gE.saveGraph()

## DBLP-ACM Datset

In [480]:
pt1 = "../data/DBLP-ACM/DBLP2.csv"
pt2 = "../data/DBLP-ACM/ACM.csv"
pt3 = "../data/DBLP-ACM/DBLP-ACM_perfectMapping.csv"

In [481]:
dblp2 = pd.read_csv(pt1, engine = 'python')
acm = pd.read_csv(pt2)
matchings = pd.read_csv(pt3)

In [482]:
display(dblp2.head())
display(acm.head())

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995


,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999


In [483]:
gE = graphEmbedder()

In [484]:
gE.defineKeys(dblp2, 'id')
gE.defineKeys(acm, 'id')
gE.defineTruth(matchings)

True

In [485]:
gE.embeddText([dblp2,acm], 'id','title','title', min_df=0)
gE.embeddText([dblp2,acm], 'id','authors','authors', min_df=0)

True

In [486]:
gE.embeddCategorical([dblp2,acm], 'id', 'venue', 'venue')
gE.embeddCategorical([dblp2,acm], 'id', 'year', 'year')

True

In [487]:
gE.saveGraph()

## DBLP-Scholar Dataset

In [488]:
pt1 = "../data/DBLP-Scholar/DBLP1.csv"
pt2 = "../data/DBLP-Scholar/Scholar.csv"
pt3 = "../data/DBLP-Scholar/DBLP-Scholar_perfectMapping.csv"

In [489]:
dblp1 = pd.read_csv(pt1, engine = 'python')
scholar = pd.read_csv(pt2)
matchings = pd.read_csv(pt3)

In [490]:
display(dblp1.head())
display(scholar.head())

,id,title,authors,venue,year
0,conf/vldb/RusinkiewiczKTWM95,Towards a Cooperative Transaction Model - The ...,"M Rusinkiewicz, W Klas, T Tesch, J W�sch, P Muth",VLDB,1995
1,journals/sigmod/EisenbergM02,SQL/XML is Making Good Progress,"A Eisenberg, J Melton",SIGMOD Record,2002
2,conf/vldb/AmmannJR95,Using Formal Methods to Reason about Semantics...,"P Ammann, S Jajodia, I Ray",VLDB,1995
3,journals/sigmod/Liu02,Editor's Notes,L Liu,SIGMOD Record,2002
4,journals/sigmod/Hammer02,Report on the ACM Fourth International Worksho...,NaN,NaN,2002


,id,title,authors,venue,year
0,aKcZKwvwbQwJ,11578 Sorrento Valley Road,QD Inc,"San Diego,",NaN
1,ixKfiTHoaDoJ,Initiation of crazes in polystyrene,"AS Argon, JG Hannoosh","Phil. Mag,",NaN
2,3BxllB4wwcIJ,Immunogold labelling is a quantitative method ...,"GH Hansen, LL Wetterberg, H SjÃ¶strÃ¶m, O NorÃ©n","The Histochemical Journal,",1992.0
3,d2WWxwKMex4J,The Burden of Infectious Disease Among Inmates...,"TM Hammett, P Harmon, W Rhodes",see,NaN
4,cZCX-AQpjccJ,The Role of Faculty Advising in Science and En...,JR Cogdell,"NEW DIRECTIONS FOR TEACHING AND LEARNING,",1995.0


In [491]:
gE = graphEmbedder()

In [492]:
gE.defineKeys(dblp1, 'id')
gE.defineKeys(scholar, 'id')
gE.defineTruth(matchings)

True

In [493]:
gE.embeddText([dblp1,scholar], 'id','title','title', min_df=0.01)
gE.embeddText([dblp1,scholar], 'id','authors','authors', min_df=0.01)
gE.embeddText([dblp1,scholar], 'id','venue','venue', min_df=0)

True

In [494]:
gE.embeddCategorical([dblp1,scholar], 'id', 'year', 'year')

True

In [495]:
gE.saveGraph()

# Amazon-Google Product Dataset

In [496]:
pt1 = "../data/Amazon-GoogleProducts/Amazon.csv"
pt2 = "../data/Amazon-GoogleProducts/GoogleProducts.csv"
pt3 = "../data/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv"

In [497]:
amazon = pd.read_csv(pt1, engine = 'python')
google = pd.read_csv(pt2, engine = 'python')
matchings = pd.read_csv(pt3)

In [498]:
display(amazon.head())
display(google.head())

,id,title,description,manufacturer,price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99


,id,name,description,manufacturer,price
0,http://www.google.com/base/feeds/snippets/1112...,learning quickbooks 2007,learning quickbooks 2007,intuit,38.99
1,http://www.google.com/base/feeds/snippets/1153...,superstart! fun with reading & writing!,fun with reading & writing! is designed to hel...,NaN,8.49
2,http://www.google.com/base/feeds/snippets/1134...,qb pos 6.0 basic software,qb pos 6.0 basic retail mngmt software. for re...,intuit,637.99
3,http://www.google.com/base/feeds/snippets/1204...,math missions: the amazing arcade adventure (g...,save spectacle city by disrupting randall unde...,NaN,12.95
4,http://www.google.com/base/feeds/snippets/1224...,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from ...,adobe software,805.99


In [499]:
google = google.rename(columns={'name':'title'})

In [500]:
google.price = pd.to_numeric(google.price.apply(fixPrice))

In [501]:
gE = graphEmbedder()

In [502]:
gE.defineKeys(amazon, 'id')
gE.defineKeys(google, 'id')
gE.defineTruth(matchings)

True

In [503]:
gE.embeddText([amazon,google], 'id','title','title', min_df=0.01)
gE.embeddText([amazon,google], 'id','description','description', min_df=0)

True

In [504]:
gE.embeddCategorical([amazon,google], 'id','manufacturer','manufacturer')

True

In [505]:
gE.embeddOrdinal([amazon,google], 'id','price','price')

True

In [506]:
gE.saveGraph()